In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# ---------------------------------------------------------
# CONFIGURATION & SEED
# ---------------------------------------------------------
np.random.seed(42)  # Ensures we get the same "random" numbers every time
random.seed(42)

NUM_CLIENTS = 500
NUM_LOANS = 750

# ---------------------------------------------------------
# DATA LISTS & WEIGHTS
# ---------------------------------------------------------
sectors = [
    'Manufacturing', 'Technology', 'Real Estate', 'Retail',
    'Energy', 'Healthcare', 'Transportation', 'Financial Services'
]
# We weigh Real Estate heavier to simulate concentration risk (Step 3, KPI #3)
sector_weights = [0.15, 0.15, 0.25, 0.10, 0.10, 0.10, 0.10, 0.05]

regions = [
    'Île-de-France', 'Auvergne-Rhône-Alpes', 'Nouvelle-Aquitaine',
    'Occitanie', 'Hauts-de-France', 'Provence-Alpes-Côte d\'Azur'
]

rms = ['RM_A', 'RM_B', 'RM_C', 'RM_D', 'RM_E']

credit_ratings = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC']
# Skewed towards BBB/BB (typical corporate portfolio)
rating_weights = [0.05, 0.10, 0.20, 0.30, 0.20, 0.10, 0.05]

products = ['Term_Loan', 'Revolving_Credit', 'Cash_Management', 'Trade_Finance', 'FX_Hedging']

# ---------------------------------------------------------
# HELPER FUNCTIONS
# ---------------------------------------------------------
def generate_client_id(index):
    return f"CORP{str(index).zfill(4)}"

def generate_loan_id(client_id, loan_num):
    return f"{client_id}_L{loan_num}"

def calculate_kyc_status(last_review_date):
    days_since_review = (datetime.now() - last_review_date).days
    if days_since_review > 730: # > 2 years
        return "Expired"
    elif days_since_review > 640: # Within 3 months of expiry
        return "Expiring Soon"
    else:
        return "Valid"

def determine_risk_flag(dpd):
    if dpd == 0: return "Performing"
    if dpd <= 14: return "Grace Period"
    if dpd <= 44: return "Watch List"
    if dpd <= 89: return "Medium Risk"
    return "High Risk"

# ---------------------------------------------------------
# GENERATE DATA
# ---------------------------------------------------------
data = []

# Generate Client Level Data
clients = []
for i in range(1, NUM_CLIENTS + 1):
    client_id = generate_client_id(i)
    sector = random.choices(sectors, weights=sector_weights)[0]
    region = random.choice(regions)
    rm = random.choice(rms)
    rating = random.choices(credit_ratings, weights=rating_weights)[0]

    # KYC Logic: Random date within last 3 years (approx 1100 days)
    days_ago = random.randint(1, 1100)
    kyc_date = datetime.now() - timedelta(days=days_ago)

    # Products Used (Random subset)
    num_products = random.randint(1, 4)
    client_products = random.sample(products, num_products)

    clients.append({
        'Client_ID': client_id,
        'Sector': sector,
        'Region': region,
        'Relationship_Manager': rm,
        'Credit_Rating': rating,
        'KYC_Last_Review_Date': kyc_date,
        'Products_Used': ",".join(client_products),
        'Product_Count': num_products
    })

# Generate Loan Level Data
for i in range(NUM_LOANS):
    # Pick a random client to attach this loan to
    client = random.choice(clients)

    loan_id = generate_loan_id(client['Client_ID'], random.randint(1, 3))
    loan_type = random.choice(['Term Loan', 'Revolving Credit'])

    # Financials
    amount = round(random.uniform(500000, 50000000), -4) # Round to nearest 10k
    # Outstanding is usually 40-100% of original amount
    outstanding = round(amount * random.uniform(0.4, 1.0), -2)

    # Dates
    start_days_ago = random.randint(100, 1500)
    origination_date = datetime.now() - timedelta(days=start_days_ago)
    duration_months = random.choice([12, 24, 36, 48, 60, 84])
    maturity_date = origination_date + timedelta(days=duration_months*30)
    months_to_maturity = (maturity_date - datetime.now()).days // 30

    # Risk Logic (Days Past Due)
    # 85% chance of 0 DPD (Performing), 15% chance of issues
    if random.random() < 0.85:
        dpd = 0
    else:
        # If late, usually 1-30 days, rarely 90+
        dpd = int(np.random.exponential(scale=20))

    # Compile Row
    row = {
        'Loan_ID': loan_id,
        'Client_ID': client['Client_ID'],
        'Relationship_Manager': client['Relationship_Manager'],
        'Loan_Type': loan_type,
        'Loan_Amount_EUR': amount,
        'Outstanding_Balance_EUR': outstanding,
        'Utilization_Rate_Pct': round(random.uniform(20, 95), 2) if loan_type == 'Revolving Credit' else 0,
        'Interest_Rate_Pct': round(random.uniform(2.5, 8.5), 2),
        'Origination_Date': origination_date.date(),
        'Maturity_Date': maturity_date.date(),
        'Months_To_Maturity': months_to_maturity,
        'Sector': client['Sector'],
        'Region': client['Region'],
        'Credit_Rating': client['Credit_Rating'],
        'Days_Past_Due': dpd,
        'Risk_Flag': determine_risk_flag(dpd),
        'KYC_Last_Review_Date': client['KYC_Last_Review_Date'].date(),
        'KYC_Status': calculate_kyc_status(client['KYC_Last_Review_Date']),
        'Products_Used': client['Products_Used'],
        'Cross_Sell_Score': int(((5 - client['Product_Count']) / 5) * 100)
    }
    data.append(row)

# ---------------------------------------------------------
# EXPORT
# ---------------------------------------------------------
df = pd.DataFrame(data)
print(f"✅ Generated {len(df)} loans for {NUM_CLIENTS} clients.")
print("Sample of the data:")
print(df.head())

# Save to CSV
filename = "banking_portfolio_data.csv"
df.to_csv(filename, index=False)
print(f"\n✅ SUCCESS: File saved as '{filename}'")
print("Check the folder icon on the left to download it!")

✅ Generated 750 loans for 500 clients.
Sample of the data:
       Loan_ID Client_ID Relationship_Manager         Loan_Type  \
0  CORP0274_L1  CORP0274                 RM_C         Term Loan   
1  CORP0403_L3  CORP0403                 RM_B  Revolving Credit   
2  CORP0274_L1  CORP0274                 RM_C         Term Loan   
3  CORP0347_L3  CORP0347                 RM_E  Revolving Credit   
4  CORP0276_L3  CORP0276                 RM_C         Term Loan   

   Loan_Amount_EUR  Outstanding_Balance_EUR  Utilization_Rate_Pct  \
0       37920000.0               28491800.0                  0.00   
1       30530000.0               17368800.0                 69.55   
2       37460000.0               32892200.0                  0.00   
3       47890000.0               19476400.0                 20.38   
4       15580000.0               13227100.0                  0.00   

   Interest_Rate_Pct Origination_Date Maturity_Date  Months_To_Maturity  \
0               3.14       2025-04-16    2028-03